# Instalacja GLPK

W pierwszej kolejności należy wykonać **instalację** GNU Linear Programming Kit, np. wg oficjalnych [instrukcji](https://www.gnu.org/software/glpk/).

Poniżej przedstawiono proces instalacji w systemie Linux na potrzeby niniejszego notatnika.

Proces kompilacji może potrwać ok 2 do 3 minut.

In [ ]:
!wget -N https://ftp.gnu.org/gnu/glpk/glpk-5.0.tar.gz
!tar -xf glpk-5.0.tar.gz
# Kompilacja i instalacja
!echo "Rozpoczynam kompilację. Może to potrwać kilka minut..."
!cd glpk-5.0/ && ./configure 1> /dev/null && make 1> /dev/null && make install 1> /dev/null
# Rejestracja biblioteki libglpk
!echo /usr/local/lib/libglpk.so.40 | tee /etc/ld.so.conf.d/libglpk.con && ldconfig
!echo "Zakończone"


# Definiujemy nowe "cell magic" dla uproszczenia uruchamiania solvera Glpsol

import re
import tempfile
import subprocess
from pathlib import Path
from IPython import get_ipython
from IPython.display import HTML, display
from IPython.core.magic import Magics, cell_magic, magics_class
import shlex
import html
import logging

logging.basicConfig(level=logging.ERROR)
logger = logging.getLogger(__name__)

HIGHLIGHT_RULES = {
    'color:green': re.compile(r'\b((?:INTEGER)?\s*OPTIMAL[\s\w]*?\n)\b', re.IGNORECASE),
    'color:red': re.compile(r'\b(INFEASIBLE|UNBOUNDED)\b', re.IGNORECASE),
    'color:#FF0000': re.compile(r'\b(MathProg model processing error)\b', re.IGNORECASE),
    'color:#c76e00': re.compile(r'((?:\bobjective:.*?\n)|(?:\bDisplay\sstatement.*?\n))', re.IGNORECASE),
}

def syntax_highlight(text):
    """
    Applies syntax highlighting to the glpsol output by wrapping matched patterns with span tags.
    """
    # Escape HTML special characters
    escaped_text = html.escape(text)

    # Apply highlighting rules
    for style, pattern in HIGHLIGHT_RULES.items():
        replacement = rf'<span style="{style}">\1</span>'
        escaped_text = pattern.sub(replacement, escaped_text)

    # Return the formatted HTML
    return f'<pre style="max-width: 800px; white-space: pre-wrap; font-family: monospace;">{escaped_text}</pre>'


@magics_class
class GlpkMagic(Magics):
    @cell_magic
    def glpsol(self, line, cell):
        """
        Cell magic to run glpsol on the provided LP model.

        Usage:
        %%glpsol [glpsol options]
        <LP model>
        """
        lp_file = None
        sol_file = None
        try:
            with tempfile.TemporaryDirectory() as tmpdir:
                tmp_path = Path(tmpdir)
                lp_file = tmp_path / "model.lp"
                sol_file = tmp_path / "model.sol"

                # Write the LP model to the temporary .lp file
                lp_file.write_text(cell, encoding='utf-8')

                # Parse command-line options safely
                cmd_options = shlex.split(line)
                cmd = ['glpsol'] + cmd_options + ['-m', str(lp_file), '-o', str(sol_file)]

                # Run glpsol and capture output
                result = subprocess.run(
                    cmd,
                    stdout=subprocess.PIPE,
                    stderr=subprocess.PIPE,
                    text=True,
                    check=True
                )

                output = result.stdout

                # Check if the solution file exists and read its content
                if sol_file.exists():
                    sol_content = sol_file.read_text(encoding='utf-8')
                    output += ('\n' + '-' * 80 + '\n') + sol_content
                else:
                    logger.warning(f"Expected solution file {sol_file} not found.")

        except subprocess.CalledProcessError as e:
            # Capture and process error output
            output = e.stdout + '\n' + e.stderr
            processed_output = self._process_error_output(output, cell, lp_file)
            highlighted = syntax_highlight(processed_output)
            display(HTML(highlighted))
            raise RuntimeError(processed_output) from None
        except Exception as e:
            # Handle unexpected exceptions
            logger.error("An unexpected error occurred while running glpsol.", exc_info=True)
            display(HTML(f'<span style="color:red;">An unexpected error occurred: {e}</span>'))
            return

        # Apply syntax highlighting and display
        highlighted = syntax_highlight(output)
        display(HTML(highlighted))

    def _process_error_output(self, output, cell, lp_file):
        """
        Processes glpsol error output to map errors back to the LP model lines.
        """
        processed = []
        input_lines = cell.split('\n')
        lp_filename = lp_file.name if lp_file else 'model.lp'

        for ln in output.split('\n'):
            if ln.startswith(f"{lp_filename}:"):
                processed.append(f'\n{ln}')
                tail = ln[len(lp_filename) + 1:]
                match = re.match(r"^\d+", tail)
                if match:
                    orig_line_index = int(match.group()) - 1
                    if 0 <= orig_line_index < len(input_lines):
                        processed.append(input_lines[orig_line_index])
            else:
                processed.append(ln)

        return '\n'.join(processed)

# Register the magic with IPython
ip = get_ipython()
if ip:
    ip.register_magics(GlpkMagic)
else:
    logger.error("IPython environment not found. The GlpkMagic was not registered.")

--2025-11-04 12:44:29--  https://ftp.gnu.org/gnu/glpk/glpk-5.0.tar.gz
Resolving ftp.gnu.org (ftp.gnu.org)... 209.51.188.20, 2001:470:142:3::b
Connecting to ftp.gnu.org (ftp.gnu.org)|209.51.188.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4132649 (3.9M) [application/x-gzip]
Saving to: ‘glpk-5.0.tar.gz’

glpk-5.0.tar.gz     100%[===================>]   3.94M  20.1MB/s    in 0.2s    

2025-11-04 12:44:29 (20.1 MB/s) - ‘glpk-5.0.tar.gz’ saved [4132649/4132649]

Rozpoczynam kompilację. Może to potrwać kilka minut...
In file included from /usr/include/string.h:535,
                 from ./env/stdc.h:35,
                 from ./env/env.h:25,
                 from misc/wclique1.c:22:
In function ‘memset’,
    inlined from ‘_glp_wclique1’ at misc/wclique1.c:111:7:
/usr/include/x86_64-linux-gnu/bits/string_fortified.h:59:10: warning: ‘__builtin_memset’ specified bound between 18446744071562067968 and 18446744073709551615 exceeds maximum object size 922337203685477

## Testowanie instalacji

W przypadku pomyślnie zakończonej instalacji poniższe wykonanie powinno skutkować wyświetleniem informacji na temat wersji pakietu GLPK, np.:
`GLPSOL--GLPK LP/MIP Solver 5.0`

In [11]:
!glpsol -v

GLPSOL--GLPK LP/MIP Solver 5.0
Copyright (C) 2000-2020 Free Software Foundation, Inc.

This program has ABSOLUTELY NO WARRANTY.

This program is free software; you may re-distribute it under the terms
of the GNU General Public License version 3 or later.


## Instalacja -- Windows

Informacje oraz pliki `*.bat` dla systemu Windows znajdują się w poniższym katalogu.

Instalacja wymaga kompilatora `Microsoft Visual C++ v14`

In [ ]:
!cd glpk-5.0/w64/ && cat readme.txt

## Dokumentacja

Oficjalna dokumentacja pakietu znajduje się w plikach `doc/glpk.pdf' oraz `doc/gmpl.pdf`.

Kopię można również pobrać, np. z fossies.org:
* [gmpl.pdf](https://fossies.org/linux/glpk/doc/gmpl.pdf) -- dokumentacja języka MathProg
* [glpk.pdf](https://fossies.org/linux/glpk/doc/glpk.pdf) -- dokumentacja solvera GLPK

# Zadanie 6

Dany jest zbiór $n$ przedmiotów $i \in \{1, 2, \ldots, n\}$ o określonej wadze $w_i$ oraz wartości $p_i$.

**Jaka jest maksymalna wartość przedmiotów, które możemy zabrać nie przekraczając limitu wagi $B$**?

*Wskazówka*. Zmienne decyzyjne powinny być binarne i oznaczać, czy przedmiot jest zabrany, czy też pozostawiony.

Znajdź odpowiedź dla następujących danych:

    param n := 3;
    param weights := [1] 10 [2] 20 [3] 30;
    param values := [1] 60 [2] 100 [3] 120;
    param B := 50;

In [15]:
%%glpsol

param B > 0;
param n, integer, > 0;
param weights {1..n};
param values {1..n};

# zmienne decyzyjne: x[i] = 1 jeśli bierzemy przedmiot i, 0 w przeciwnym razie
var x {i in 1..n}, binary;

# funkcja celu — maksymalizujemy sumę wartości
maximize TotalValue:
    sum {i in 1..n} values[i] * x[i];

# ograniczenie wagowe
s.t. WeightLimit:
    sum {i in 1..n} weights[i] * x[i] <= B;

solve;

# wypisanie wyników
printf "\nMaksymalna wartość: %g\n", TotalValue;
printf "Wybrane przedmioty:\n";
for {i in 1..n} {
    if x[i] > 0.5 then
        printf "Przedmiot %d (waga=%d, wartość=%d)\n", i, weights[i], values[i];
}

data;

param n := 3;
param weights :=
  1  10
  2  20
  3  30;
param values :=
  1 60
  2 100
  3 120;
param B := 50;

end;


UsageError: Cell magic `%%glpsol` not found.


# Zadanie 6b

Rozwiąż instancje problemu plecakowego (ang. Knapsack problem) ze strony:

https://people.sc.fsu.edu/~jburkardt/datasets/knapsack_01/knapsack_01.html

Rozwiąż wszystkie (8 zestawów) używając zewnętrznych plików z danymi. Sprawdź poprawność rozwiązań.

# Zadanie 7

Załóżmy, że dla danej macierzy `NxN` należy wybrać **N** liczb, tak aby ich suma była największa, ale każda wybrana liczba powinna być *jedyną* wybraną w swoim wierszu i swojej kolumnie. Na przykład dla macierzy:

      7  53 183 439
    627 343 773 959
    447 283 463  29
    217 623   3 399

można osiągnąć maksymalną sumę wynoszącą 2282 wybierając z kolejnych wierszy: 439, 773, 447 i 623.

Napisz program całkowitoliczbowy rozwiązujący problem.

*Wskazówka*. W celu zapisania macierzy, możemy zdefiniować, np. parametr `param numbers {1..n, 1..n};` , którego wartości określamy
w sekcji (lub pliku) z danymi jako:

    param numbers:
        1   2   3   4 :=
    1   7  53 183 439
    2 627 343 773 959
    3 447 283 463  29
    4 217 623   3 399 ;


Jaka jest odpowiedź (suma) dla następujących danych:


    param numbers:
         1    2    3    4    5    6    7    8    9   10   11   12   13   14   15 :=
     1   7   53  183  439  863  497  383  563   79  973  287   63  343  169  583
     2 627  343  773  959  943  767  473  103  699  303  957  703  583  639  913
     3 447  283  463   29   23  487  463  993  119  883  327  493  423  159  743
     4 217  623    3  399  853  407  103  983   89  463  290  516  212  462  350
     5 960  376  682  962  300  780  486  502  912  800  250  346  172  812  350
     6 870  456  192  162  593  473  915   45  989  873  823  965  425  329  803
     7 973  965  905  919  133  673  665  235  509  613  673  815  165  992  326
     8 322  148  972  962  286  255  941  541  265  323  925  281  601   95  973
     9 445  721   11  525  473   65  511  164  138  672   18  428  154  448  848
    10 414  456  310  312  798  104  566  520  302  248  694  976  430  392  198
    11 184  829  373  181  631  101  969  613  840  740  778  458  284  760  390
    12 821  461  843  513   17  901  711  993  293  157  274   94  192  156  574
    13  34  124    4  878  450  476  712  914  838  669  875  299  823  329  699
    14 815  559  813  459  522  788  168  586  966  232  308  833  251  631  107
    15 813  883  451  509  615   77  281  613  459  205  380  274  302   35  805 ;

# Zadanie 8

System obsługi serwerowni zarządza alokacją zasobów obliczeniowych (CPU oraz RAM) dla maszyn wirtualnych (VM) działających
na serwerach w serwerowni.
Zadanie polega na optymalizacji przydziału
zasobów, tak aby zminimalizować zużycie energii jednocześnie zapewniając żądany poziom zasobów każdej VM.

Założenia:
* każda maszyna VM potrzebuje określonej mocy CPU oraz ilości RAM do efektywnego działania
* każdy serwer oferuje określoną moc CPU oraz wielość RAM
* każdy serwer ma zdefiniowane maksymalne zużycie energii podczas działania -- w przypadku *częściowego* obciążenia zużycie jest *proporcjonalne* do wykorzystywanej mocy obliczeniowej CPU, np. serwer o mocy 1000 W i udostępniający moc CPU równą 800 zużyje 500 W, jeżeli działajace na nim VM zużywają łącznie 400 jednostek mocy CPU (czyli połowę).
* każda VM może zostać uruchomiona tylko na 1 serwerze
* łączne zużycie energii przez wszystkie serwery nie może przekroczyć masymalnej wartości `MaxPower`

_Wskazówka_. Proszę przyjrzeć się problemowi skojarzenia (agentów z zadaniami) omawianemu podczas wykładu.

Zaproponuj model rozwiązujący powyższy problem. Jaka jest odpowiedź dla następujących danych:


    param MaxPower := 400;  /* Granica łącznej wykorzystywanej mocy */
    
    set Server := 1 2 3 ;  /* id serwerów */
    set VM := 1 2 3 4 5 ;  /* id maszyn wirt. */
    
    /* Moc serwerów przy pełnym obciążeniu: */
    param Power :=   
    1 100
    2 150
    3 120 ;
    
    /* Zasoby serwerów */
    param : ServerCPU ServerRAM :=
    1   200   2048
    2   300   4096
    3   250   3072 ;
    
    /* Wymagania maszyn wirt. */
    param :
      VM_CPU  VM_RAM :=
    1     50     512
    2     60    1024
    3     40     768
    4    150    1024
    5     85     512 ;

# Zadanie 9

 Liczba 1210 jest przykładem liczby **autobiograficznej**, gdyż jej pierwsza cyfra jest zarazem liczbą zer w jej zapisie dziesiętnym, druga - liczbą jedynek i tak dalej, aż do ostatniej, która jest liczbą trójek.

Przykładowo, 7-cyfrową liczbą autobiograficzną jest 3211000, ponieważ ma 3 zera, 2 jedynki, 1 dwójkę, 1 trójkę, 0 czwórek, 0 piątek i 0 szóstek.

Napisz program całokowitoliczbowy, który znajdzie 10-cyfrową liczbę autobiograficzną.

*Wskazówka. Rozważ użycie zmiennych binarnych przypisujących każdej pozycji w liczbie dokładnie jedną cyfrę.*

# Zadanie 10

Liczba 153 jest przykładem liczby **specjalnej**, ponieważ suma jej cyfr podniesionych do potęgi równej liczbie cyfr daje samą liczbę:

$ 1^3 + 5^3 + 3^3 = 1 + 125 + 27 = 153 $

Innym przykładem jest liczba 9474:

$ 9^4 + 4^4 + 7^4 + 4^4 = 6561 + 256 + 2401 + 256 = 9474 $

Napisz program całkowitoliczbowy dla solwera GNU GLPK, który znajdzie najmniejszą 7-cyfrową liczbę _specjalną_.